# Supervised Learning: The Ice Cream Prediction Engine 

Welcome to the **TMU Intro AI class**. Today, you aren't just a student; you are the owner of a startup: **The Ice Cream Engine**.

**The Problem**: You waste money if you buy too much ice cream (it melts) or too little (lost sales).

**The Physics of Business**:
1.  **Customers (Quantity)** depend on the **Temperature** (Hotter = More people).
2.  **Customers (Quantity)** also depend on your **Price** (Higher Price = Fewer people).
3.  **Revenue** = Price $\times$ Quantity.

**Your Goal**: Train a machine to predict **Quantity Sold** so you can optimize your stock and revenue.

---


In [1]:
# --- PROFESSOR'S TOOLS ---
# Please run this cell to initialize your lab environment.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, FloatSlider, IntSlider, Checkbox, Button, Output, VBox, HBox, Label
import ipywidgets as widgets
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
import os

# Determine a consistent aesthetic for our plots
plt.style.use('seaborn-v0_8-darkgrid')
COLORS = {'data': '#2E86C1', 'model': '#E74C3C', 'error': '#95A5A6'}

# --- DATA GENERATION (REALISTIC PHYSICS) ---
# We strictly model Quantity (Q) first, then derived Revenue (R)
# Q = f(Temp) - f(Price)
def generate_ice_cream_data():
    np.random.seed(42)
    n_samples = 100
    
    # 1. Simple Data: Price is constant ($5.00), varies only with Temp
    # Customers = Base + 4 per degree
    temp = np.random.uniform(10, 35, n_samples)
    quantity = 4 * temp + 10 + np.random.normal(0, 10, n_samples)
    # Add some outliers (bus load of tourists)
    quantity[::20] += 50 
    df_simple = pd.DataFrame({'Temperature_C': np.round(temp, 1), 'Quantity_Sold': np.round(quantity, 0)})
    
    # 2. Polynomial Data: Extremely hot days reduce sales
    temp_poly = np.random.uniform(5, 40, n_samples)
    q_poly = -0.2 * (temp_poly - 28)**2 + 150 + np.random.normal(0, 10, n_samples)
    df_poly = pd.DataFrame({'Temperature_C': np.round(temp_poly, 1), 'Quantity_Sold': np.round(np.maximum(q_poly, 0), 0)})
    
    # 3. Multivariate Data: Temp AND Price vary
    # Quantity goes UP with Temp, DOWN with Price
    temp_m = np.random.uniform(15, 35, n_samples)
    price_m = np.random.uniform(2, 10, n_samples)
    
    # The Law of Demand: High price kills demand
    q_multi = (5 * temp_m) - (15 * price_m) + 50 + np.random.normal(0, 10, n_samples)
    q_multi = np.maximum(q_multi, 0) # Can't have negative sales
    
    revenue_m = q_multi * price_m

    df_multi = pd.DataFrame({
        'Temperature_C': np.round(temp_m, 1), 
        'Price_USD': np.round(price_m, 2), 
        'Quantity_Sold': np.round(q_multi, 0),
        'Revenue_USD': np.round(revenue_m, 2)
    })
    
    return df_simple, df_poly, df_multi

# Generate data directly in memory
df_simple, df_poly, df_multi = generate_ice_cream_data()

X_simple = df_simple['Temperature_C'].values
y_simple = df_simple['Quantity_Sold'].values

print("✅ Lab Environment Initialized. Data Generated Successfully.")
df_simple.head()

✅ Lab Environment Initialized. Data Generated Successfully.


,Temperature_C,Quantity_Sold
0,19.4,138.0
1,33.8,142.0
2,28.3,124.0
3,25.0,90.0
4,13.9,63.0


## Part 1: The Intuition (Predicting Crowd Size)

Before thinking about money, let's think about **people**.
Look at the data below. **Hotter days = More People (Quantity Sold)**.

### Interactive Challenge
You are the model. Adjust the **Slope** ($w$) and **Bias** ($b$) to predict how many ice creams you will sell based on the temperature.

> **Note**: Ideally, we want the red error lines to be zero.

In [2]:
def plot_interactive_model(w, b):
    plt.figure(figsize=(10, 6))
    
    # 1. Plot Data
    plt.scatter(X_simple, y_simple, color=COLORS['data'], s=50, label='Actual Sales (Qty)', alpha=0.7)
    
    # 2. Plot Your Model
    y_pred = w * X_simple + b
    plt.plot(X_simple, y_pred, color=COLORS['model'], linewidth=3, label=f'Model: {w:.1f} * Temp + {b:.0f}')
    
    # 3. Visualize Errors (Residuals)
    loss = 0
    for i in range(len(X_simple)):
        plt.plot([X_simple[i], X_simple[i]], [y_simple[i], y_pred[i]], color=COLORS['error'], linewidth=1, alpha=0.5)
        loss += (y_simple[i] - y_pred[i])**2
    
    mse = loss / len(X_simple)
    
    plt.title(f'Human Regression | MSE Loss: {mse:.2f}', fontsize=16)
    plt.xlabel('Temperature (°C)', fontsize=12)
    plt.ylabel('Quantity Sold (Units)', fontsize=12)
    plt.legend()
    plt.show()

interact(plot_interactive_model, 
         w=FloatSlider(min=0, max=10, step=0.1, value=4, description='Slope (People/Deg)'), 
         b=FloatSlider(min=-50, max=100, step=5, value=10, description='Base People'));

interactive(children=(FloatSlider(value=4.0, description='Slope (People/Deg)', max=10.0), FloatSlider(value=10…

## Part 2: Opening the Black Box (The Loss Landscape)

You just manually minimized the error. But how does a machine do it with no eyes?

It treats the "Error" as a terrain. 
- Imagine a valley where the **lowest point** represents the **perfect model**.
- The machine is a ball dropped somewhere on this terrain.
- Gravity pulls it down. That gravity is **Gradient Descent**.

### Visualization: The Valley of Error
Below is a 3D view of the error for different combinations of Slope ($w$) and Bias ($b$).
- **X-axis**: Slope
- **Y-axis**: Bias
- **Z-axis (Height)**: Error (High is bad, Low is good)

Watch how the red ball (your model) rolls down to the bottom.

In [3]:
def plot_loss_landscape(view_angle=45):
    # Create a grid of w and b values
    w_vals = np.linspace(0, 10, 50)
    b_vals = np.linspace(-50, 100, 50)
    W, B = np.meshgrid(w_vals, b_vals)
    Z = np.zeros_like(W)
    
    # Compute MSE for every combination (Brute force for visualization)
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            pred = W[i, j] * X_simple + B[i, j]
            Z[i, j] = np.mean((y_simple - pred)**2)

    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Surface plot
    surf = ax.plot_surface(W, B, Z, cmap='viridis', alpha=0.8, edgecolor='none')
    
    # True Optimal point (found via Linear Algebra)
    # We cheat slightly here to show the goal
    true_w, true_b = np.polyfit(X_simple, y_simple, 1)
    min_z = np.mean((y_simple - (true_w * X_simple + true_b))**2)
    
    ax.scatter([true_w], [true_b], [min_z], color='red', s=100, label='Global Minimum (Best Model)')

    ax.set_xlabel('Slope (w)')
    ax.set_ylabel('Bias (b)')
    ax.set_zlabel('Error (MSE)')
    ax.set_title('The Loss Landscape: Finding the Lowest Valley')
    ax.view_init(elev=30, azim=view_angle)
    plt.legend()
    plt.show()

interact(plot_loss_landscape, view_angle=IntSlider(min=0, max=360, step=10, value=45, description='Rotate View'));

interactive(children=(IntSlider(value=45, description='Rotate View', max=360, step=10), Output()), _dom_classe…

## Part 3: The Math from Scratch

Now we implement the "Gravity" that pulls the ball down.

**The Equation**: $w_{new} = w_{old} - \alpha \cdot \frac{\partial Error}{\partial w}$

- $\alpha$ (Alpha) is the **Learning Rate** (Step size).
- $\frac{\partial Error}{\partial w}$ is the **Gradient** (Slope of the hill). 

If the gradient is positive (uphill), we subtract it to go downhill.

### Interactive Gradient Descent
Play with the `learning_rate`.
- **Small (0.0001)**: The model learns slowly. Safe but boring.
- **Good (0.001)**: The model converges quickly.
- **Too High (0.01)**: The model overshoots and explodes! (Watch the error go to infinity).

In [4]:
def train_and_visualize(learning_rate=0.0005, iterations=50):
    # 1. Initialize Randomly
    w = 0.0
    b = 0.0
    n = len(X_simple)
    
    history_w = [w]
    history_b = [b]
    history_loss = []
    
    # 2. Training Loop
    for i in range(iterations):
        y_pred = w * X_simple + b
        loss = np.mean((y_simple - y_pred)**2)
        history_loss.append(loss)
        
        # Gradients (Derivatives)
        dw = (-2/n) * np.sum(X_simple * (y_simple - y_pred))
        db = (-2/n) * np.sum(y_simple - y_pred)
        
        # Update
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        history_w.append(w)
        history_b.append(b)
        
    # 3. Visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: The Model fitting process
    ax1.scatter(X_simple, y_simple, color=COLORS['data'], alpha=0.5)
    ax1.plot(X_simple, history_w[0]*X_simple + history_b[0], 'k--', label='Start', alpha=0.5)
    ax1.plot(X_simple, history_w[-1]*X_simple + history_b[-1], 'r-', linewidth=3, label='Final')
    ax1.set_title(f'Model Evolution (Iter {iterations})')
    ax1.legend()
    
    # Plot 2: Learning Curve
    ax2.plot(history_loss, linewidth=2)
    ax2.set_title('Learning Curve (Loss over Time)')
    ax2.set_xlabel('Iteration')
    ax2.set_ylabel('MSE Loss')
    ax2.grid(True)
    
    print(f"Final Loss: {history_loss[-1]:.2f} | Final Slope (w): {w:.2f} | Final Bias (b): {b:.2f}")
    plt.show()

interact(train_and_visualize,
         learning_rate=FloatSlider(min=0.0001, max=0.002, step=0.0001, value=0.0005, description='Learning Rate', readout_format='.4f'),
         iterations=IntSlider(min=10, max=200, step=10, value=50, description='Iterations'));

interactive(children=(FloatSlider(value=0.0005, description='Learning Rate', max=0.002, min=0.0001, readout_fo…

## Part 4: Complexity (Polynomials & Overfitting)

**New Data**: The world isn't always linear. In our new dataset, sales drop if it gets **too hot** (people stay indoors).

A straight line ($y=mx+b$) fits this poorly. We need curves.
We use **Polynomial Regression** by adding powers of $x$ ($x^2, x^3...$) as new features.

### The Trap: Overfitting
If we use too much power (Degree 20), we can fit every single point perfectly, but the model becomes useless.

- **Degree 1 (Linear)**: Too simple (Underfitting).
- **Degree 2 (Quadratic)**: Just right.
- **Degree 15**: Madness (Overfitting). Fits the noise, not the pattern.

In [5]:
# Use the complex data we already generated in memory
X_poly = df_poly[['Temperature_C']].values # Keep as 2D array for sklearn
y_poly = df_poly['Quantity_Sold'].values

def plot_polynomial_fit(degree=2):
    plt.figure(figsize=(12, 6))
    
    # 1. Transform Features (Add powers of x)
    poly = PolynomialFeatures(degree=degree)
    X_poly_features = poly.fit_transform(X_poly)
    
    # 2. Train Linear Regression on transformed features
    model = LinearRegression()
    model.fit(X_poly_features, y_poly)
    
    # 3. Predict on a smooth curve range to visualize wiggles
    X_range = np.linspace(X_poly.min(), X_poly.max(), 100).reshape(-1, 1)
    X_range_poly = poly.transform(X_range)
    y_range_pred = model.predict(X_range_poly)
    
    # Visualization
    plt.scatter(X_poly, y_poly, color=COLORS['data'], alpha=0.6, label='Data')
    plt.plot(X_range, y_range_pred, color=COLORS['model'], linewidth=2, label=f'Polynomial Degree {degree}')
    
    plt.title(f'Polynomial Regression (Degree {degree})')
    plt.xlabel('Temperature')
    plt.ylabel('Quantity Sold')
    plt.ylim(0, 300) 
    plt.legend()
    plt.show()

interact(plot_polynomial_fit, 
         degree=IntSlider(min=1, max=20, step=1, value=2, description='Polynomial Degree'));

interactive(children=(IntSlider(value=2, description='Polynomial Degree', max=20, min=1), Output()), _dom_clas…

## Part 5: The Real World (Multivariate Analysis)

In the real world, you control the **Price**. 

**Economic Physics**:
1.  **More Heat** = More Customers (+)
2.  **Higher Price** = Fewer Customers (-)

We will train a model to predict **Quantity** (Crowd Size), and then multiply by Price to see Revenue.

In [6]:
# Use the multivariate data we already generated in memory
X_multi = df_multi[['Temperature_C', 'Price_USD']].values # 2 inputs
y_multi = df_multi['Quantity_Sold'].values # Predicting PEOPLE, not money yet

# 1. Pre-Train the model
final_model = LinearRegression()
final_model.fit(X_multi, y_multi)

def plot_3d_regression(elev=20, azim=45):
    # Create a meshgrid for the plane
    x1_range = np.linspace(X_multi[:,0].min(), X_multi[:,0].max(), 10)
    x2_range = np.linspace(X_multi[:,1].min(), X_multi[:,1].max(), 10)
    X1, X2 = np.meshgrid(x1_range, x2_range)
    
    # Predict Z (Quantity) for the plane
    X_grid = np.array([X1.flatten(), X2.flatten()]).T
    Z = final_model.predict(X_grid).reshape(X1.shape)

    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Plot Data
    ax.scatter(X_multi[:,0], X_multi[:,1], y_multi, c=COLORS['data'], alpha=0.8, s=30, label='Actual Quantity')
    
    # Plot Plane
    ax.plot_surface(X1, X2, Z, alpha=0.4, color=COLORS['model'])
    
    ax.set_xlabel('Temperature (C)')
    ax.set_ylabel('Price ($)')
    ax.set_zlabel('Quantity Sold')
    ax.set_title('Predicting Customer Count (Quantity)')
    
    ax.view_init(elev=elev, azim=azim)
    plt.show()
    
    print(f"Model: Quantity = {final_model.coef_[0]:.2f}*Temp + {final_model.coef_[1]:.2f}*Price + {final_model.intercept_:.2f}")
    print("Notice: Price coefficient is negative! (Raising price scares people away).")

interact(plot_3d_regression, 
         elev=IntSlider(min=0, max=90, step=5, value=20, description='Elevation'), 
         azim=IntSlider(min=0, max=360, step=5, value=45, description='Rotation (Azimuth)'));

interactive(children=(IntSlider(value=20, description='Elevation', max=90, step=5), IntSlider(value=45, descri…

## Part 6: Deployment (Optimization)

**The Ultimate Question**: 
Tomorrow is forecasted to be **30°C**. What price should you set to maximize profit?

- Set Price too Low? Lots of customers, but tiny margins.
- Set Price too High? No customers, zero revenue.

Use the dashboard to find the "Sweet Spot".

In [7]:
# Interactive Prediction Dashboard
temp_input = FloatSlider(min=10, max=40, step=1, value=30, description='Forecast Temp:')
price_input = FloatSlider(min=1, max=15, step=0.5, value=5, description='Your Price:')
output_q = Label(value="Predicted Customers: --")
output_r = Label(value="Predicted Revenue: $ --")

def update_prediction(temp, price):
    # 1. Predict Quantity using AI
    predicted_qty = final_model.predict([[temp, price]])[0]
    predicted_qty = max(0, predicted_qty) # Can't have negative people
    
    # 2. Calculate Revenue using Math
    predicted_revenue = predicted_qty * price
    
    output_q.value = f"Predicted Customers: {int(predicted_qty)} people"
    output_r.value = f"Predicted Revenue: ${predicted_revenue:.2f}"

out = widgets.interactive_output(update_prediction, {'temp': temp_input, 'price': price_input})

display(VBox([
    Label("--- Ice Cream Strategy Dashboard ---"), 
    temp_input, 
    price_input, 
    Label("--- AI Prediction ---"),
    output_q, 
    output_r
]))